### 230620更新: 重新編排 syscall 受詞的類別，共 9 類
- ['NET', 'PROC', 'MEM', 'ID', 'FILE', 'TIME', 'RESOURCE_BYTE', 'EXIT_STATUS', 'PERMISSION']
- 這個版本沒有 device，歸類為 file。
- 似乎有些小問題(志剛標的類別)，像是 pipe() 和 pipe2() 的類別不一樣，一個是 file 一個是 proc，但沒有樣本用到所以沒差。

In [11]:
import pandas as pd

synbase = pd.read_csv('../synonym_data/synonym_dataset.csv')
print(synbase['Obj_type'].unique())
syscall_type_dict = {}
for i, row in synbase.iterrows():
    syscall_type_dict[row['Syscall']] = row['Obj_type']
print(f"num of syscall {len(syscall_type_dict)} (include 'rm()', 'exec()')")

# 更改 395 syscall rule 的類別
# rules = pd.read_csv('../rule_dataset_final.csv')
# for syscall, type in syscall_type_dict.items():
#     # print(syscall)
#     rules.loc[rules['Syscall'] == syscall, ['EntityType']] = type
# rules.to_csv('../rule_dataset_final.csv', index=False)
# rules.to_csv('./rule_dataset_final.csv', index=False)

# 更改正在使用的 rule 的類別
rules = pd.read_csv('../../Model/rule_dataset.csv')
for syscall, type in syscall_type_dict.items():
    rules.loc[rules['Syscall'] == syscall, ['EntityType']] = type
rules.to_csv('../../Model/rule_dataset.csv', index=False)

['NET' 'PROC' 'MEM' 'ID' 'FILE' 'TIME' 'RESOURCE_BYTE' 'EXIT_STATUS'
 'PERMISSION']
num of syscall 397 (include 'rm()', 'exec()')


### Load synonym data and current rule data
- ps: 分析用的規則是 `rule_dataset_final.csv`，包含完整的 system calls。實作上沒有用。
- `synonym_dataset.csv`

In [7]:
import pandas as pd

rules = pd.read_csv('../rule_dataset_final.csv')

def clean_braces(x):
    x = x.replace("()", "").replace(",", "").replace(" ", "")
    return str(x)
def add_braces(x):
    x = clean_braces(x)
    x = x+"()"
    return x

rules['Syscall'] = rules['Syscall'].apply(add_braces) #clean_braces
rules.head(3)

,EntityType,ActionType,Source,Syscall,EnVerb,Sentence
0,DEVICE,DEVICE,man,io_cancel(),cancel,NaN
1,DEVICE,DEVICE,man,io_destroy(),destroy,NaN
2,DEVICE,DEVICE,man,io_getevents(),read,NaN


In [8]:
synbase = pd.read_csv('../synonym_data/synonym_dataset.csv')
synbase = synbase[~synbase['Syscall'].isin(['rm()', 'exec()'])]
display(synbase.head(3))

# 只有一個 Synonym 的 syscall: 115 個
def count_pairs(x):
    numOfSplit = len(x.split('|'))
    if numOfSplit == 1 and (x == '' or x == '<,>'):
        return 0
    return numOfSplit

# dataset['numOfSynonym'] = dataset['Synonym'].apply(count_pairs)
# dataset['numOfFirstLine'] = dataset['FirstLine'].apply(count_pairs)

def extract_verb(x:str):
    ''' <,> => None, <get,xxx> => get '''
    x = x.replace('<','').replace('>','')
    x = x.split(',')[0]
    return x.strip()

# dataset['FirstVerb'] = dataset['FirstLine'].apply(extract_verb)

,Syscall,Sentence,FirstLine,Synonym,SynonymSentence
0,shutdown(),shut down part of a full-duplex connection,"<shut down,part of a full-duplex connection>","<,>",NaN
1,recvmmsg(),receive multiple messages on a socket,"<receive,multiple messages>","<,>",NaN
2,sendmmsg(),send multiple messages on a socket,"<send,multiple messages>","<transmit,multiple messages>",transmit multiple messages on a socket using a...


### Extract verbs and sentence from synbase

In [21]:
syndata = {}
counter = 0
syscall_list = set()
for i, synrow in synbase.iterrows():
    verbs = set()
    # print(synrow)
    # extract_verb(synrow['FirstLine'])
    verb_name = extract_verb(synrow['FirstLine'])
    verbs.add(extract_verb(synrow['FirstLine'])) # name sector
    counter += 1
    des = []
    if synrow['Synonym'] != '<,>':
        des = synrow['Synonym'].split('|') # description sector
        des = [extract_verb(d) for d in des]
        [verbs.add(extract_verb(d)) for d in des]
    syscall = synrow['Syscall']
    # if syscall in syscall_list:
    #     print(f"duplicate syscall: {syscall}")
    syscall_list.add(syscall)
    counter += len(des)
    if verb_name in des:
        print(f"{syscall} has same verb as verb_name and description: {verb_name}") # , {des}
        continue
    if len(des) != len(set(des)):
        print(f"{syscall} has duplicate description verb: {des}")
    if len(verbs) == 0:
        continue
    try:
        # syndata[syscall].update(verbs)
        syndata[syscall].extend(list(verbs))
    except:
        # syndata[syscall] = set(verbs)
        syndata[syscall] = list(verbs)

from itertools import islice
def take(n, iterable):
    """Return the first n items of the iterable as a list."""
    return list(islice(iterable, n))

n_items = take(5, syndata.items())
n_items

ioperm() has same verb as verb_name and description: set
mlockall() has same verb as verb_name and description: lock
init_module() has same verb as verb_name and description: load
finit_module() has same verb as verb_name and description: load
mknod() has same verb as verb_name and description: create
timerfd_settime() has same verb as verb_name and description: arm
timerfd_settime() has same verb as verb_name and description: disarm
fchmodat() has same verb as verb_name and description: change
timerfd_create() has same verb as verb_name and description: create
open_by_handle_at() has same verb as verb_name and description: open
userfaultfd() has same verb as verb_name and description: create


[('shutdown()', ['shut down']),
 ('recvmmsg()', ['receive']),
 ('sendmmsg()', ['transmit', 'send']),
 ('pipe2()', ['create']),
 ('mq_unlink()', ['remove'])]

In [32]:
dup = synbase.duplicated()
dup = [1 for x in dup if x]
sum(dup)

0

In [22]:
counter
# syndata

583

依照 synbase 建立新的 rule dataset

In [35]:
def getType(syscall:str, ruledf:pd.DataFrame):
    ''' search in rule dataset and get 'EntityType', 'ActionType'.'''
    entity, action = None, None
    rules = ruledf[ruledf['Syscall'] == syscall]
    if len(rules) != 0:
        rules = rules.reset_index()
        # display(rules)
        entity = rules.loc[0, 'EntityType']
        action = rules.loc[0, 'ActionType']
    return entity, action

getType('sendmmsg()', rules)

('NET', 'NET_TRANSMIT')

In [36]:
ruledata = []
for syscall, verbset in syndata.items():
    entity, action = getType(syscall, rules)
    for verb in verbset:
        ruledata.append({
            'EntityType': entity,
            'ActionType': action,
            'Source': 'man', 
            'Syscall': syscall, 
            'EnVerb': verb, 
            'Sentence': None,
        })
df = pd.DataFrame(ruledata) # df is the new rule dataset for statistic
print('num of syscall', len(df['Syscall'].unique()),', num of verb',  len(df['EnVerb'].unique()))
df

num of syscall 395 , num of verb 136


,EntityType,ActionType,Source,Syscall,EnVerb,Sentence
0,NET,NET_CLOSE,man,shutdown(),shut down,None
1,NET,NET_TRANSMIT,man,recvmmsg(),receive,None
2,NET,NET_TRANSMIT,man,sendmmsg(),transmit,None
3,NET,NET_TRANSMIT,man,sendmmsg(),send,None
4,NET,NET_CONNECT,man,pipe2(),create,None
...,...,...,...,...,...,...
563,NET,NET_TRANSMIT,man,recvfrom(),receive,None
564,NET,NET_TRANSMIT,man,send(),send,None
565,NET,NET_TRANSMIT,man,sendto(),send,None
566,NET,NET_TRANSMIT,man,sendmsg(),send,None


In [3]:
# 輸出 csv 並標註
# df.to_csv('need_label.csv', index=False) 
# 重新 sort 規則
df = pd.read_csv('need_label.csv')
df = df.sort_values(by=['EntityType', 'ActionType', 'Syscall'])
df.to_csv('after_label.csv', index=False) 

### add verb from CTI report back to rule dataet

- 現在新的 rule_dataset 已經建好了(從 synbase 把手冊翻譯轉過來)，把從 CTI 找到的動詞加過去

In [5]:
df =pd.read_csv('after_label.csv') 
rules_from_cti = rules[rules['Source'] != 'man']
df = pd.concat([df, rules_from_cti])
df = df.sort_values(by=['EntityType', 'ActionType', 'Syscall'])

print('num of syscall', len(df['Syscall'].unique()),', num of verb',  len(df['EnVerb'].unique()))
df.head()

num of syscall 396 , num of verb 155


,EntityType,ActionType,Source,Syscall,EnVerb,Sentence
0,DEVICE,DEVICE,man,io_cancel(),cancel,NaN
1,DEVICE,DEVICE,man,io_destroy(),destroy,NaN
2,DEVICE,DEVICE,man,io_getevents(),read,NaN
3,DEVICE,DEVICE,man,io_setup(),create,NaN
4,DEVICE,DEVICE,man,io_submit(),submit,NaN


In [6]:
df.to_csv('rule_dataset_final.csv', index=False)

- Next step，需要人工刪除 from 手冊和 from CTI 重複的動詞
- 例如說 nanosleep() 的 sleep 動詞在手冊跟 CTI 都有重複 => 刪除 CTI 的這行